In [1]:
import os
# Add directory for NVIDIA gpu. Ignore if not Windows
os.add_dll_directory("C:/Program Files/NVIDIA GPU Computing Toolkit/CUDA/v11.2/bin")

<AddedDllDirectory('C:/Program Files/NVIDIA GPU Computing Toolkit/CUDA/v11.2/bin')>

In [3]:
import tensorflow as tf
# print(tf.reduce_sum(tf.random.normal([1000, 1000])))
# Confirm tensorflow running on gpu
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [22]:
from tensorflow import keras
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import Resizing
import numpy as np
import pickle as pkl

### Base CovNet Model

In [5]:
# function to build the conv net base


# complete this function
def build_base_convnet_model():
    """Re-create the model from the first prompt, but with a different input shape"""
    
    # Return this variable
    model = None
    
    # YOUR CODE HERE
    inputs = keras.Input(shape = (299, 299, 3))
    x = keras.layers.Conv2D(filters = 32, kernel_size = 3, activation = 'relu')(inputs)
    x = keras.layers.MaxPooling2D(pool_size = 2)(x)
    x = keras.layers.Conv2D(filters = 64, kernel_size = 3, activation = 'relu')(x)
    x = keras.layers.MaxPooling2D(pool_size = 2)(x)
    x = keras.layers.Conv2D(filters = 128, kernel_size = 3, activation = 'relu')(x)
    x = keras.layers.Flatten()(x)
    outputs = keras.layers.Dense(1, activation = 'sigmoid')(x)

    model = keras.Model(inputs, outputs)

    model.compile(loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

def load_image_data(base_path: str) -> tuple:
    """Write a function that accepts a base path that contains all of the directories, and creates training,
    validation and test sets"""
    
    # Return these variables from the function
    train_data = keras.utils.image_dataset_from_directory(f'{base_path}/train', 
                                                          image_size = (299, 299),
                                                          batch_size = 32)

    validation_data = keras.utils.image_dataset_from_directory(f'{base_path}/val', 
                                                          image_size = (299, 299),
                                                          batch_size = 32)
    
    test_data = keras.utils.image_dataset_from_directory(f'{base_path}/test', 
                                                          image_size = (299, 299),
                                                          batch_size = 32)
    
    # YOUR CODE HERE
    
    
    return train_data, validation_data, test_data

def fit_convnet_model(model, train_set, validation_set):
    """Fit a model with the above stated criteria"""
    early_stopping = keras.callbacks.EarlyStopping(patience = 10)
    
    # YOUR CODE HERE
    model.fit(train_set, 
              validation_data = validation_set, 
              callbacks = [early_stopping], 
              epochs = 500)
    
    return model

In [6]:
# run this cell to create your base convolution model and training, validation & test sets
base_mod = build_base_convnet_model()
train_data, validation_data, test_data = load_image_data('../data/COVID-19_Radiography_Dataset/TwoClasses/split')

Found 8284 files belonging to 2 classes.
Found 2761 files belonging to 2 classes.
Found 2763 files belonging to 2 classes.


In [7]:
fitted_model = fit_convnet_model(base_mod, train_data, validation_data)

Epoch 1/500
259/259 [==============================] - 26s 81ms/step - loss: 35.5139 - accuracy: 0.7575 - val_loss: 0.3734 - val_accuracy: 0.8566
Epoch 2/500
259/259 [==============================] - 20s 76ms/step - loss: 16.5739 - accuracy: 0.8143 - val_loss: 0.3903 - val_accuracy: 0.8577
Epoch 3/500
259/259 [==============================] - 20s 77ms/step - loss: 6.3294 - accuracy: 0.8543 - val_loss: 3.6633 - val_accuracy: 0.7816
Epoch 4/500
259/259 [==============================] - 20s 76ms/step - loss: 9.4186 - accuracy: 0.8799 - val_loss: 0.6813 - val_accuracy: 0.8816
Epoch 5/500
259/259 [==============================] - 20s 77ms/step - loss: 16.8079 - accuracy: 0.8841 - val_loss: 5.7160 - val_accuracy: 0.8823
Epoch 6/500
259/259 [==============================] - 20s 76ms/step - loss: 3.8142 - accuracy: 0.9141 - val_loss: 1.3226 - val_accuracy: 0.8957
Epoch 7/500
259/259 [==============================] - 20s 75ms/step - loss: 2.0736 - accuracy: 0.9192 - val_loss: 0.9410 - val

### Inception V3 Model



In [9]:
from keras.applications import InceptionV3

In [19]:
def build_transfer_learning_model():
    
    # return this variable
    model = None

    model_input = keras.Input(shape = (299, 299, 3))
    
    base_model = InceptionV3(input_shape = (299, 299, 3), weights='imagenet', include_top=False)

    # make the weights in the base model non-trainable
    for layer in base_model.layers:
      layer.trainable = False

    # combine the base model with a dense layer and output layer for the 10 classes
    # the preprocess_input transforms input data according to how the model was trained
    
    x = keras.applications.inception_v3.preprocess_input(model_input)
    x = base_model(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(256, activation = 'relu')(x)
    x = keras.layers.Dropout(0.5)(x)
    output = keras.layers.Dense(1, activation = 'sigmoid')(x)

    model = keras.Model(model_input, output)

    model.compile(loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [20]:
transfer_learning_mod = build_transfer_learning_model()

In [21]:
inception_model = fit_convnet_model(transfer_learning_mod, train_data, validation_data)

Epoch 1/500
259/259 [==============================] - 31s 106ms/step - loss: 5.1110 - accuracy: 0.8396 - val_loss: 0.2213 - val_accuracy: 0.9174
Epoch 2/500
259/259 [==============================] - 27s 104ms/step - loss: 0.4943 - accuracy: 0.8950 - val_loss: 0.3729 - val_accuracy: 0.9203
Epoch 3/500
259/259 [==============================] - 27s 102ms/step - loss: 0.3215 - accuracy: 0.9229 - val_loss: 0.1975 - val_accuracy: 0.9453
Epoch 4/500
259/259 [==============================] - 27s 104ms/step - loss: 0.2826 - accuracy: 0.9326 - val_loss: 0.3630 - val_accuracy: 0.9185
Epoch 5/500
259/259 [==============================] - 27s 103ms/step - loss: 0.2693 - accuracy: 0.9419 - val_loss: 0.1521 - val_accuracy: 0.9580
Epoch 6/500
259/259 [==============================] - 27s 103ms/step - loss: 0.1734 - accuracy: 0.9541 - val_loss: 0.1605 - val_accuracy: 0.9616
Epoch 7/500
259/259 [==============================] - 27s 103ms/step - loss: 0.1665 - accuracy: 0.9545 - val_loss: 0.1694 -

### ResNet Model

In [ ]:
from keras.applications import ResNet50

In [19]:
def build_transfer_learning_model():
    
    # return this variable
    model = None

    model_input = keras.Input(shape = (299, 299, 3))
    
    base_model = ResNet50(input_shape = (299, 299, 3), weights='imagenet', include_top=False)

    # make the weights in the base model non-trainable
    for layer in base_model.layers:
      layer.trainable = False

    # combine the base model with a dense layer and output layer for the 10 classes
    # the preprocess_input transforms input data according to how the model was trained
    
    x = keras.applications.resnet50.preprocess_input(model_input)
    x = base_model(x)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dense(256, activation = 'relu')(x)
    x = keras.layers.Dropout(0.5)(x)
    output = keras.layers.Dense(1, activation = 'sigmoid')(x)

    model = keras.Model(model_input, output)

    model.compile(loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [20]:
transfer_learning_mod = build_transfer_learning_model()

In [21]:
resnet_model = fit_convnet_model(transfer_learning_mod, train_data, validation_data)

Epoch 1/500
259/259 [==============================] - 31s 106ms/step - loss: 5.1110 - accuracy: 0.8396 - val_loss: 0.2213 - val_accuracy: 0.9174
Epoch 2/500
259/259 [==============================] - 27s 104ms/step - loss: 0.4943 - accuracy: 0.8950 - val_loss: 0.3729 - val_accuracy: 0.9203
Epoch 3/500
259/259 [==============================] - 27s 102ms/step - loss: 0.3215 - accuracy: 0.9229 - val_loss: 0.1975 - val_accuracy: 0.9453
Epoch 4/500
259/259 [==============================] - 27s 104ms/step - loss: 0.2826 - accuracy: 0.9326 - val_loss: 0.3630 - val_accuracy: 0.9185
Epoch 5/500
259/259 [==============================] - 27s 103ms/step - loss: 0.2693 - accuracy: 0.9419 - val_loss: 0.1521 - val_accuracy: 0.9580
Epoch 6/500
259/259 [==============================] - 27s 103ms/step - loss: 0.1734 - accuracy: 0.9541 - val_loss: 0.1605 - val_accuracy: 0.9616
Epoch 7/500
259/259 [==============================] - 27s 103ms/step - loss: 0.1665 - accuracy: 0.9545 - val_loss: 0.1694 -